In [ ]:
from importlib import reload
from helper import run
import ecc
import helper

from ecc import FieldElement, Point

In [ ]:
a = FieldElement(num=0, prime=223)
b = FieldElement(num=7, prime=223)
x = FieldElement(num=192, prime=223)
y = FieldElement(num=105, prime=223)
p1 = Point(x, y, a, b)
print(p1)

In [ ]:
reload(ecc)
run(ecc.ECCTest('test_on_curve'))

In [ ]:
prime = 223
a = FieldElement(num=0, prime=prime)
b = FieldElement(num=7, prime=prime)
x1 = FieldElement(num=192, prime=prime)
y1 = FieldElement(num=105, prime=prime)
x2 = FieldElement(num=17, prime=prime)
y2 = FieldElement(num=56, prime=prime)
p1 = Point(x1, y1, a, b)
p2 = Point(x2, y2, a, b)
print(p1+p2)
#Point(170,142)_0_7 FieldElement(223)

In [ ]:
# F223 
# (170,142) + (60,139)
# Canonical form is y^{2}=x^{3}+ax+b
prime = 223
a = FieldElement(num=0, prime=prime)
b = FieldElement(num=7, prime=prime)
x1 = FieldElement(num=170, prime=prime)
y1 = FieldElement(num=142, prime=prime)
x2 = FieldElement(num=60, prime=prime)
y2 = FieldElement(num=139, prime=prime)
p1 = Point(x1, y1, a, b)
p2 = Point(x2, y2, a, b)
print(p1+p2)


In [ ]:
# Scalar multiplication

reload(ecc)
prime = 223
a = FieldElement(0, prime)
b = FieldElement(7, prime)

x = FieldElement(47, prime)
y = FieldElement(71, prime)
p = Point(x, y, a, b)
for s in range(1,21):
    result = s*p
    print('{}*(47,71)=({},{})'.format(s,result.x.num,result.y.num))



### Groups

Combining Fields with Elliptic Curves bring us to methematical Groups.

* For public key cryptography we want finite cyclic groups
* We can take a generator point from an elliptic curve over a field group and generate a finite cyclic group
* Groups have a single operation - point addition

Group Properties

* Identity - A + 0 = A
    * Identity is the point at infinity
* Closure - aG +bG = (a+b)G
* Invertability - If aG is in the group, (n-a)G is in the group
* Commutivity - aG + bG = bG + aG
* Associativity - aG + (bG + cG) = (aG + bG) + cG

In [ ]:
# Order of group
reload(ecc)
prime = 223
a = FieldElement(0, prime)
b = FieldElement(7, prime)
x = FieldElement(15, prime)
y = FieldElement(86, prime)
p = Point(x, y, a, b)
inf = Point(None, None, a, b)
product = p
count = 1
print(product)
while product != inf:
    product += p
    print(product)
    count += 1
print(count)



### Curve for Cryptography

Defining a curve for public key cryptography involves

* Specifying the a and b of the curve  $y^{2}=x^{3}+ax+b$
* Specify the prime of the finite field
* Specify the x and y of the generator point G
* Specify the order of the group genated by G, n

For bitcoin:

* a = 0, b = 7, making the equation $y^{2} = x3 + 7$
* $p = 2^{256} – 2^{32} – 977$
* $G_x$ = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
* $G_y$ = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
* n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141


In [ ]:
p = 2**256 - 2**32 - 977
gx = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [ ]:
# Is G on the curve?
print(gy**2 % p == (gx**3 + 7) % p)

In [ ]:
# Does the generator point G have order n?
x = FieldElement(gx, p)
y = FieldElement(gy, p)
seven = FieldElement(7, p)
zero = FieldElement(0, p)
G = Point(x, y, zero, seven)
print(n*G)

### Public Key Cryptography

Public key cryptography takes advantage of the discrete log properties of scalar multiplication of points on an elliptic curve over a field: there is a method to multiply a point by a scalar, but because of the non-linear properties of point addition there is not defined way to perform division.

The key operation is P = eG - we can easily compute P knowing e and G, but we cannot easily compute e when we know P and G. In public key cryptography, P is the public key, and e is the private key. In this set up, e is a 256-bit number, and P is a coordinate (x,y) where x and y ads 256 bit numbers.


### Signature Algorithm

ECDSA - Elliptic Curve Digital Signature Algorithm

* The secret is the e satisfying eG = P
* The target in the book;s william tell analogy is a random number k, and target it like kG = R, and we only care about the x coordinate of R, denoted as r
* The following is equivalent to the discrete log problem, where u and v != 0 can be selected by the signer and P and G are known: uG + vP = kG
    * The above implies vP = G(v - u)
    * Since v is not equal to zero we can divide to get P = ((k - u)/v)G
    * If we know e then we can get eG =  ((k - u)/v)G
    * If you can solve uG + vP = kG with some (u,v) you either know e or have broken the discrete log problem
    
* The purpose of the shooting needs to also be incorporated - in general this is a signature hash, denoted by z.
* This is incorporated into uG + vP as u = z/s, v = r/s
* uG + vP = R = kG
* uG = veG = kG
* u + ve = k
* z/s + re/s = k
* s = (z + re)/k


Verification is easy

* With v not equal to zero uG + vP = (z/s)G + (re/s)G = = ((z+re)/s)G = (z+re)/((z+re)/k))G = kG = R = (r,y) 
    

In [ ]:
# Signature validation

reload(ecc)
from ecc import S256Point, G, N

z = 0xbc62d4b80d9e36da29c16c5d4d9f11731f36052c72401a76c23c0fb5a9b74423
r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec
px = 0x04519fac3d910ca7e7138f7013706f619fa8f033e6ec6e09370ea38cee6a7574
py = 0x82b51eab8c27c66e26c858a079bcdf4f1ada34cec420cafc7eac1a42216fb6c4
point = S256Point(px, py)
s_inv = pow(s, N-2, N)  # Fermat's little theorem for calc inverse this way

u = z * s_inv % N
v = r * s_inv % N  
print((u*G + v*point).x.num == r)

In [ ]:
# Signature gen

from ecc import S256Point, G, N
from helper import hash256
e = int.from_bytes(hash256(b'my secret'), 'big')  
z = int.from_bytes(hash256(b'my message'), 'big')  
k = 1234567890  
r = (k*G).x.num  
k_inv = pow(k, N-2, N)
s = (z+r*e) * k_inv % N  
point = e*G  
print(point)
print(hex(z))
print(hex(r))
print(hex(s))



Exerise 7

Sign the following message with the secret:

```
e = 12345
z = int.from_bytes(hash256('Programming Bitcoin!'), 'big')
```

In [ ]:
import helper
e = 12345
z = int.from_bytes(hash256(b'Programming Bitcoin!'), 'big')

In [ ]:
from ecc import S256Point, G, N
from helper import hash256
e = 12345
z = int.from_bytes(hash256(b'Programming Bitcoin!'), 'big')
k = 1234567890
r = (k*G).x.num
k_inv = pow(k, N-2, N)
s = (z+r*e) * k_inv % N
print(e*G)



In [ ]:
reload(ecc)
from ecc import PrivateKey
pk = PrivateKey(e)

In [ ]:
sig = pk.sign(z)
print(sig)

In [ ]:
p = e*G

In [ ]:
p.verify(z,sig)